In [97]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


For running and mounting gdrive or downloading csv file used by me use this link
https://docs.google.com/spreadsheets/d/1uCI6fwtGgU9N5ZZbhY-YLdgmdjdXeFRgFjxQHOkpnng/edit?usp=sharing

In [152]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import nltk
import time
nltk.download('stopwords')
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from nltk.tokenize import word_tokenize, sent_tokenize 
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
stop_words = set(stopwords.words('english')) 


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [153]:
dataset_imdb=pd.read_csv('/content/gdrive/MyDrive/spam.csv',encoding='latin-1')

In [154]:
dataset_imdb.head(10)

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN
5,spam,FreeMsg Hey there darling it's been 3 week's n...,NaN,NaN,NaN
6,ham,Even my brother is not like to speak with me. ...,NaN,NaN,NaN
7,ham,As per your request 'Melle Melle (Oru Minnamin...,NaN,NaN,NaN
8,spam,WINNER!! As a valued network customer you have...,NaN,NaN,NaN
9,spam,Had your mobile 11 months or more? U R entitle...,NaN,NaN,NaN


In [155]:
#Text before any manipulation
dataset_imdb['v2'].iloc[0]

'Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...'

2) Remove punctuations, special characters and stopwords from the text in ‘SMS’ column. Convert the text to lower case. 

In [156]:
#Let us do the removal of puncuations,special characters and stop words from only 1st row for testing.

#1) Preprocessing and modifying data 


def preprocessText(text):
   text=BeautifulSoup(text, "html.parser").get_text()
   text=re.sub('\[[^]]*\]', ' ',text)
   text=re.sub('[^a-zA-Z]', ' ',text)
   text=text.lower()
   return text
   

dataset_imdb['v2']=dataset_imdb['v2'].apply(lambda s:s.lower() if type(s) == str else s)

dataset_imdb['v2']=dataset_imdb['v2'].apply(lambda s:BeautifulSoup(s, "html.parser").get_text() if type(s) == str else s)
dataset_imdb['v2']=dataset_imdb['v2'].apply(lambda s:re.sub('\[[^]]*\]', ' ',s) if type(s) == str else s)
dataset_imdb['v2']=dataset_imdb['v2'].apply(lambda s:re.sub('[^a-zA-Z]', ' ',s) if type(s) == str else s)
#dataset_imdb['v2'].apply(lambda cw :preprocessText(cw))
#dataset_imdb['sms']



dataset_imdb['v2']=dataset_imdb['v2'].apply(lambda  x: [item for item in x.split() if item not in stop_words ])
dataset_imdb['v2']=dataset_imdb['v2'].apply(lambda  x: ' '.join(x))

dataset_imdb['v2']


0       go jurong point crazy available bugis n great ...
1                                 ok lar joking wif u oni
2       free entry wkly comp win fa cup final tkts st ...
3                     u dun say early hor u c already say
4                  nah think goes usf lives around though
                              ...                        
5567    nd time tried contact u u pound prize claim ea...
5568                            b going esplanade fr home
5569                                pity mood suggestions
5570    guy bitching acted like interested buying some...
5571                                       rofl true name
Name: v2, Length: 5572, dtype: object

In [157]:
dataset_imdb['v2']

0       go jurong point crazy available bugis n great ...
1                                 ok lar joking wif u oni
2       free entry wkly comp win fa cup final tkts st ...
3                     u dun say early hor u c already say
4                  nah think goes usf lives around though
                              ...                        
5567    nd time tried contact u u pound prize claim ea...
5568                            b going esplanade fr home
5569                                pity mood suggestions
5570    guy bitching acted like interested buying some...
5571                                       rofl true name
Name: v2, Length: 5572, dtype: object

Create two objects X and y. X will be the 'SMS' column of data frame and y will be the 'label' (spam or ham )

In [158]:
X=dataset_imdb['v2']
#Converting sentiment column Positive:1 Negative : 0
dataset_imdb['v1'] = (dataset_imdb['v1'].replace({'ham': 1, 'spam': 0})).values
Y=dataset_imdb['v1']

Create a CountVectorizer object and split the data into training and testing sets. Train a MultinomialNB model and Display the confusion Matrix

In [159]:
spam_dataset_train, spam_dataset_test, train_data_label, test_data_label = train_test_split(X,Y, test_size=0.25, random_state=42)
spam_dataset_train.shape


(4179,)

In [160]:
spam_dataset_test.shape

(1393,)

In [161]:
#CountVectorizer object with ngram range from 1 to 3 and setting binary to false
count_vec1 = CountVectorizer(ngram_range=(1, 3), binary=False)
#CountVectorizer object with ngram range from 1 to 3 and setting binary to true
count_vec2 = CountVectorizer(ngram_range=(1, 3), binary=True)
#CountVectorizer object with no parameter
count_vec3 = CountVectorizer()

In [162]:
count_vec_train1 = count_vec1.fit_transform(spam_dataset_train)
count_vec_test1 = count_vec1.transform(spam_dataset_test)

count_vec_train2 = count_vec2.fit_transform(spam_dataset_train)
count_vec_test2 = count_vec2.transform(spam_dataset_test)

count_vec_train3 = count_vec3.fit_transform(spam_dataset_train)
count_vec_test3 = count_vec3.transform(spam_dataset_test)

In [163]:
#MultinomialNB model
multi_clf = MultinomialNB()

In [164]:
#Predicting with CountVectorizer object with ngram range from 1 to 3 and setting binary to false
multi_clf.fit(count_vec_train1, train_data_label)

predict_NB1 = multi_clf.predict(count_vec_test1)

In [165]:
#Predicting with CountVectorizer object with ngram range from 1 to 3 and setting binary to true

multi_clf.fit(count_vec_train2, train_data_label)

predict_NB2 = multi_clf.predict(count_vec_test2)

In [166]:
#Predicting with CountVectorizer object with no parameters

multi_clf.fit(count_vec_train3, train_data_label)

predict_NB3 = multi_clf.predict(count_vec_test3)

In [167]:
#Displaying the Confusion Matrix,Classification report and Accuracy for 1st model
print("Spam Dataset Classification Report: \n", classification_report(test_data_label, predict_NB1,target_names=['Spam','Ham']))
print("Spam Dataset Confusion Matrix: \n", confusion_matrix(test_data_label, predict_NB1))
print("Accuracy: \n", accuracy_score(test_data_label, predict_NB1))

Spam Dataset Classification Report: 
               precision    recall  f1-score   support

        Spam       0.97      0.91      0.94       191
         Ham       0.99      1.00      0.99      1202

    accuracy                           0.98      1393
   macro avg       0.98      0.95      0.96      1393
weighted avg       0.98      0.98      0.98      1393

Spam Dataset Confusion Matrix: 
 [[ 173   18]
 [   5 1197]]
Accuracy: 
 0.9834888729361091


In [168]:
#Displaying the Confusion Matrix,Classification report and Accuracy for 2nd model
print("Spam Dataset Classification Report: \n", classification_report(test_data_label, predict_NB2,target_names=['Spam','Ham']))
print("Spam Dataset Confusion Matrix: \n", confusion_matrix(test_data_label, predict_NB2))
print("Accuracy: \n", accuracy_score(test_data_label, predict_NB2))

Spam Dataset Classification Report: 
               precision    recall  f1-score   support

        Spam       0.97      0.90      0.93       191
         Ham       0.98      1.00      0.99      1202

    accuracy                           0.98      1393
   macro avg       0.98      0.95      0.96      1393
weighted avg       0.98      0.98      0.98      1393

Spam Dataset Confusion Matrix: 
 [[ 171   20]
 [   5 1197]]
Accuracy: 
 0.9820531227566404


In [169]:
#Displaying the Confusion Matrix,Classification report and Accuracy for 3rd model
print("Spam Dataset Classification Report: \n", classification_report(test_data_label, predict_NB3,target_names=['Spam','Ham']))
print("Spam DatasetConfusion Matrix: \n", confusion_matrix(test_data_label, predict_NB3))
print("Accuracy: \n", accuracy_score(test_data_label, predict_NB3))

Spam Dataset Classification Report: 
               precision    recall  f1-score   support

        Spam       0.95      0.91      0.93       191
         Ham       0.99      0.99      0.99      1202

    accuracy                           0.98      1393
   macro avg       0.97      0.95      0.96      1393
weighted avg       0.98      0.98      0.98      1393

Spam DatasetConfusion Matrix: 
 [[ 174   17]
 [   9 1193]]
Accuracy: 
 0.9813352476669059


Display the POS tagging on the first 4 rows of ‘SMS’

In [170]:
#For first 4 rows of original IMDB dataset
for i in range(0,4):
  text_tokenized = sent_tokenize(dataset_imdb['v2'].iloc[i])
  print(dataset_imdb['v2'].iloc[i])
  print('Row Number: ',i)
  for i in text_tokenized: 
      tokenize_word_list = nltk.word_tokenize(i) 
      tokenize_word_list = [w for w in tokenize_word_list if not w in stop_words]  
      POS_TAG = nltk.pos_tag(tokenize_word_list) 
      print(POS_TAG) 

go jurong point crazy available bugis n great world la e buffet cine got amore wat
Row Number:  0
[('go', 'VB'), ('jurong', 'JJ'), ('point', 'NN'), ('crazy', 'NN'), ('available', 'JJ'), ('bugis', 'NN'), ('n', 'RB'), ('great', 'JJ'), ('world', 'NN'), ('la', 'NN'), ('e', 'VBP'), ('buffet', 'JJ'), ('cine', 'NN'), ('got', 'VBD'), ('amore', 'RB'), ('wat', 'JJ')]
ok lar joking wif u oni
Row Number:  1
[('ok', 'JJ'), ('lar', 'JJ'), ('joking', 'NN'), ('wif', 'NN'), ('u', 'JJ'), ('oni', 'NN')]
free entry wkly comp win fa cup final tkts st may text fa receive entry question std txt rate c apply
Row Number:  2
[('free', 'JJ'), ('entry', 'NN'), ('wkly', 'VBD'), ('comp', 'NN'), ('win', 'NN'), ('fa', 'JJ'), ('cup', 'VBZ'), ('final', 'JJ'), ('tkts', 'NN'), ('st', 'NN'), ('may', 'MD'), ('text', 'VB'), ('fa', 'JJ'), ('receive', 'JJ'), ('entry', 'NN'), ('question', 'NN'), ('std', 'VBD'), ('txt', 'JJ'), ('rate', 'NN'), ('c', 'NNS'), ('apply', 'VBP')]
u dun say early hor u c already say
Row Number:  3
[('

Build and display a dependency parser  tree for the sentence : “Sansa, the second child of Ned and Catelyn, remained captive in King's Land ing after her father was executed.”

In [171]:
#We used SPACY to display dependency parser tree
import spacy
nlp = spacy.load('en_core_web_sm')
from spacy import displacy

In [172]:
#Feeding sample text given in the spacy function.

sample_text = "Sansa, the second child of Ned and Catelyn, remained captive in King's Land ing after her father was executed."

sample_sentence = nlp(sample_text)

#Printing the tokens text,tag
for token in sample_sentence:
  print (token.text, token.tag_, token.head.text, token.dep_)

Sansa NNP remained nsubj
, , Sansa punct
the DT child det
second JJ child amod
child NN Sansa appos
of IN child prep
Ned NNP of pobj
and CC Ned cc
Catelyn NNP Ned conj
, , Sansa punct
remained VBD remained ROOT
captive JJ remained acomp
in IN captive prep
King NNP Land poss
's POS King case
Land NNP in pobj
ing NN remained advcl
after IN executed mark
her PRP$ father poss
father NN executed nsubjpass
was VBD executed auxpass
executed VBN remained advcl
. . remained punct
